local:

In [50]:
import itertools
import numpy as np

def matrix(a, b, match_score=1, gap_cost=1):
    H = np.zeros((len(a) + 1, len(b) + 1), np.int)

    for i, j in itertools.product(range(1, H.shape[0]), range(1, H.shape[1])):
        match = H[i - 1, j - 1] + (match_score if a[i - 1] == b[j - 1] else - match_score)
        delete = H[i - 1, j] - gap_cost
        insert = H[i, j - 1] - gap_cost
        H[i, j] = max(match, delete, insert, 0)
    return H

def traceback(H, b, b_='', old_i=0):
    # flip H to get index of **last** occurrence of H.max() with np.argmax()
    H_flip = np.flip(np.flip(H, 0), 1)
    i_, j_ = np.unravel_index(H_flip.argmax(), H_flip.shape)
    i, j = np.subtract(H.shape, (i_ + 1, j_ + 1))  # (i, j) are **last** indexes of H.max()
    if H[i, j] == 0:
        return b_, i, j
    b_ = b[j - 1] + '-' + b_ if old_i - i > 1 else b[j - 1] + b_
    return traceback(H[0:i, 0:j], b, b_, i)

def local_alignment(a, b):
    H = matrix(a, b)
    b_, i, j = traceback(H, b)
    matches = len(b_.replace("-", ""))
    b_ = b[:j]+b_+b[j+matches:]
    if j > i:
        a = a.rjust(j-i+len(a), '-').ljust(len(b_), '-')
        b_ = b_.ljust(len(a), '-')
    else :
        b_ = b_.rjust(i-j+len(b_), '-').ljust(len(a), '-')
        a = a.ljust(len(b_), '-')
    print(a)
    print(b_)

local_alignment('BBBCCCBBBBBBAAAAAAABBBCCCC', 'BBBCCCCCCCCCCAAAAAABBBCCCC')

BBBCCCBBBBBBAAAAAAABBBCCCC-
BBBCCC-ABBBCCCCAAAAABBBCCCC


global:

In [49]:
%%bash
./t weights2.csv BBBCCCBBBBBBAAAAAAABBBCCCC BBBCCCCCCCCCCAAAAAABBBCCCC

weights:
map[AA:1 AB:-1 AC:-1 A_:-1 BA:-1 BB:1 BC:-1 B_:-1 CA:-1 CB:-1 CC:1 C_:-1 _A:-1 _B:-1 _C:-1]
aligned sequences:
BBBCCCBBBBBBAAAAAAABBBCCCC
BBBCCCCCCCCCCAAAAAABBBCCCC
